In [ ]:
HTML(read(open("style.css"), String))

# Simple Evaluation Function

This notebook can return a static centipawn value for each chess position. It will not look into the future and only consider the current state of the board. 

This method has been developed by Tomasz Michniewski in the Polish chess programming discussion list (progszach).


https://www.chessprogramming.org/Simplified_Evaluation_Function

In [ ]:
using Pkg
#Pkg.add("Chess")
using Chess
using Traceur

In [ ]:
if @isdefined(pawn_square_table) == false
    const pawn_square_table::Array{Array{Int8, 1}, 1} = [
        [0,  0,  0,  0,  0,  0,  0,  0],
        [50, 50, 50, 50, 50, 50, 50, 50],
        [10, 10, 20, 30, 30, 20, 10, 10],
        [5,  5, 10, 25, 25, 10,  5,  5],
        [0,  0,  0, 20, 20,  0,  0,  0],
        [5, -5,-10,  0,  0,-10, -5,  5],
        [5, 10, 10,-20,-20, 10, 10,  5],
        [0,  0,  0,  0,  0,  0,  0,  0]
    ]

    const knight_square_table::Array{Array{Int8, 1}, 1} = [
        [-50,-40,-30,-30,-30,-30,-40,-50],
        [-40,-20,  0,  0,  0,  0,-20,-40],
        [-30,  0, 10, 15, 15, 10,  0,-30],
        [-30,  5, 15, 20, 20, 15,  5,-30],
        [-30,  0, 15, 20, 20, 15,  0,-30],
        [-30,  5, 10, 15, 15, 10,  5,-30],
        [-40,-20,  0,  5,  5,  0,-20,-40],
        [-50,-40,-30,-30,-30,-30,-40,-50]
    ]
    
    const bishop_square_table::Array{Array{Int8, 1}, 1} = [
        [-20,-10,-10,-10,-10,-10,-10,-20],
        [-10,  0,  0,  0,  0,  0,  0,-10],
        [-10,  0,  5, 10, 10,  5,  0,-10],
        [-10,  5,  5, 10, 10,  5,  5,-10],
        [-10,  0, 10, 10, 10, 10,  0,-10],
        [-10, 10, 10, 10, 10, 10, 10,-10],
        [-10,  5,  0,  0,  0,  0,  5,-10],
        [-20,-10,-10,-10,-10,-10,-10,-20]
    ]

    const rook_square_table::Array{Array{Int8, 1}, 1} = [
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 5, 10, 10, 10, 10, 10, 10,  5],
        [-5,  0,  0,  0,  0,  0,  0, -5],
        [-5,  0,  0,  0,  0,  0,  0, -5],
        [-5,  0,  0,  0,  0,  0,  0, -5],
        [-5,  0,  0,  0,  0,  0,  0, -5],
        [-5,  0,  0,  0,  0,  0,  0, -5],
        [ 0,  0,  0,  5,  5,  0,  0,  0]
    ]

    const queen_square_table::Array{Array{Int8, 1}, 1} = [
        [-20,-10,-10, -5, -5,-10,-10,-20],
        [-10,  0,  0,  0,  0,  0,  0,-10],
        [-10,  0,  5,  5,  5,  5,  0,-10],
        [ -5,  0,  5,  5,  5,  5,  0, -5],
        [  0,  0,  5,  5,  5,  5,  0, -5],
        [-10,  5,  5,  5,  5,  5,  0,-10],
        [-10,  0,  5,  0,  0,  0,  0,-10],
        [-20,-10,-10, -5, -5,-10,-10,-20]
    ]

    const king_square_middle_game_table::Array{Array{Int8, 1}, 1} = [
        [-30,-40,-40,-50,-50,-40,-40,-30],
        [-30,-40,-40,-50,-50,-40,-40,-30],
        [-30,-40,-40,-50,-50,-40,-40,-30],
        [-30,-40,-40,-50,-50,-40,-40,-30],
        [-20,-30,-30,-40,-40,-30,-30,-20],
        [-10,-20,-20,-20,-20,-20,-20,-10],
        [ 20, 20,  0,  0,  0,  0, 20, 20],
        [ 20, 30, 10,  0,  0, 10, 30, 20]
    ]

    const king_square_end_game_table::Array{Array{Int8, 1}, 1} = [
        [-50,-40,-30,-20,-20,-30,-40,-50],
        [-30,-20,-10,  0,  0,-10,-20,-30],
        [-30,-10, 20, 30, 30, 20,-10,-30],
        [-30,-10, 30, 40, 40, 30,-10,-30],
        [-30,-10, 30, 40, 40, 30,-10,-30],
        [-30,-10, 20, 30, 30, 20,-10,-30],
        [-30,-30,  0,  0,  0,  0,-30,-30],
        [-50,-30,-30,-30,-30,-30,-30,-50]
    ]
end

More efficient solution for get_Intex_by_file

In [ ]:
if @isdefined(PIECE_VALUES) == false
    const PIECE_VALUES::Dict{PieceType, Int} = Dict([(PAWN, 100),(KNIGHT, 320),(BISHOP, 330),(ROOK, 500),(QUEEN, 900),(KING, 20000)])
end

In [ ]:
function get_value(piece::Piece)::Int64
    pieceType = ptype(piece)
    if pieceType in keys(PIECE_VALUES)
        return PIECE_VALUES[pieceType]
    end
end

According to the article:
"Additionally we should define where the ending begins. For me it might be either if:

Both sides have no queens or
Every side which has a queen has additionally no other pieces or one minorpiece maximum."

In [ ]:
function is_endgame(board::Board)::Bool
    # Inline Functions to calculate the number of minor pieces
    num_minor_pieces(side) = squarecount(knights(board, side)) + squarecount(bishops(board, side))
    # Check if Player White or Black is in endgame
    is_in_endgame(color) = isempty(queens(board, color)) || (num_minor_pieces(color) <= 1 && isempty(rooks(board, color)))
    return is_in_endgame(WHITE) && is_in_endgame(BLACK)
end

## Get_Square_Value

The function `get_square_value` is used to evaluate the position of a chess piece on a board. As input parameters it takes a board (board::Board), a chess piece (piece::Piece) and a playing field (square::Square) on which the piece is located. The function calculates a numeric centipawn value that describes the strength or weakness of the current position of the piece. 

The King uses two tables. The `table_opt` can be set to "end" to enforce the usage of the endgame table. This feature overrides the actual position of the board.

In [ ]:
if @isdefined(WHITE_PIECE_SQUARE_TABLES) == false
    const WHITE_PIECE_SQUARE_TABLES::Dict{PieceType, Vector{Vector{Int8}}} = Dict(
        PAWN => [
        [0,  0,  0,  0,  0,  0,  0,  0],
        [50, 50, 50, 50, 50, 50, 50, 50],
        [10, 10, 20, 30, 30, 20, 10, 10],
        [5,  5, 10, 25, 25, 10,  5,  5],
        [0,  0,  0, 20, 20,  0,  0,  0],
        [5, -5,-10,  0,  0,-10, -5,  5],
        [5, 10, 10,-20,-20, 10, 10,  5],
        [0,  0,  0,  0,  0,  0,  0,  0]
    ],
        KNIGHT => [
        [-50,-40,-30,-30,-30,-30,-40,-50],
        [-40,-20,  0,  0,  0,  0,-20,-40],
        [-30,  0, 10, 15, 15, 10,  0,-30],
        [-30,  5, 15, 20, 20, 15,  5,-30],
        [-30,  0, 15, 20, 20, 15,  0,-30],
        [-30,  5, 10, 15, 15, 10,  5,-30],
        [-40,-20,  0,  5,  5,  0,-20,-40],
        [-50,-40,-30,-30,-30,-30,-40,-50]
    ],
        BISHOP => [
        [-20,-10,-10,-10,-10,-10,-10,-20],
        [-10,  0,  0,  0,  0,  0,  0,-10],
        [-10,  0,  5, 10, 10,  5,  0,-10],
        [-10,  5,  5, 10, 10,  5,  5,-10],
        [-10,  0, 10, 10, 10, 10,  0,-10],
        [-10, 10, 10, 10, 10, 10, 10,-10],
        [-10,  5,  0,  0,  0,  0,  5,-10],
        [-20,-10,-10,-10,-10,-10,-10,-20]
    ],
        ROOK => [
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 5, 10, 10, 10, 10, 10, 10,  5],
        [-5,  0,  0,  0,  0,  0,  0, -5],
        [-5,  0,  0,  0,  0,  0,  0, -5],
        [-5,  0,  0,  0,  0,  0,  0, -5],
        [-5,  0,  0,  0,  0,  0,  0, -5],
        [-5,  0,  0,  0,  0,  0,  0, -5],
        [ 0,  0,  0,  5,  5,  0,  0,  0]
    ],
        QUEEN => [
        [-20,-10,-10, -5, -5,-10,-10,-20],
        [-10,  0,  0,  0,  0,  0,  0,-10],
        [-10,  0,  5,  5,  5,  5,  0,-10],
        [ -5,  0,  5,  5,  5,  5,  0, -5],
        [  0,  0,  5,  5,  5,  5,  0, -5],
        [-10,  5,  5,  5,  5,  5,  0,-10],
        [-10,  0,  5,  0,  0,  0,  0,-10],
        [-20,-10,-10, -5, -5,-10,-10,-20]
    ],
        KING => [
        [-30,-40,-40,-50,-50,-40,-40,-30],
        [-30,-40,-40,-50,-50,-40,-40,-30],
        [-30,-40,-40,-50,-50,-40,-40,-30],
        [-30,-40,-40,-50,-50,-40,-40,-30],
        [-20,-30,-30,-40,-40,-30,-30,-20],
        [-10,-20,-20,-20,-20,-20,-20,-10],
        [ 20, 20,  0,  0,  0,  0, 20, 20],
        [ 20, 30, 10,  0,  0, 10, 30, 20]
    ]
    )
    const white_king_square_end_game_table::Array{Array{Int8, 1}, 1} = [
        [-50,-40,-30,-20,-20,-30,-40,-50],
        [-30,-20,-10,  0,  0,-10,-20,-30],
        [-30,-10, 20, 30, 30, 20,-10,-30],
        [-30,-10, 30, 40, 40, 30,-10,-30],
        [-30,-10, 30, 40, 40, 30,-10,-30],
        [-30,-10, 20, 30, 30, 20,-10,-30],
        [-30,-30,  0,  0,  0,  0,-30,-30],
        [-50,-30,-30,-30,-30,-30,-30,-50]
    ]
end

In [ ]:
if @isdefined(BLACK_PIECE_SQUARE_TABLES) == false
    const BLACK_PIECE_SQUARE_TABLES::Dict{PieceType, Vector{Vector{Int8}}} = Dict(
        PAWN => [
        [0,  0,  0,   0,   0,  0,  0, 0],
        [5,  10, 10, -20, -20, 10, 10, 5],
        [5, -5, -10,  0,   0, -10, -5, 5],
        [0,  0,   0, 20, 20, 0, 0, 0],
        [5, 5, 10, 25, 25, 10, 5, 5],
        [10, 10, 20, 30, 30, 20, 10, 10],
        [50, 50, 50, 50, 50, 50, 50, 50],
        [0, 0, 0, 0, 0, 0, 0, 0]
    ],
        KNIGHT => [
        [-50, -40, -30, -30, -30, -30, -40, -50],
        [-40, -20, 0, 5, 5, 0, -20, -40],
        [-30, 5, 10, 15, 15, 10, 5, -30],
        [-30, 0, 15, 20, 20, 15, 0, -30],
        [-30, 5, 15, 20, 20, 15, 5, -30],
        [-30, 0, 10, 15, 15, 10, 0, -30],
        [-40, -20, 0, 0, 0, 0, -20, -40],
        [-50, -40, -30, -30, -30, -30, -40, -50]
    ],
        BISHOP => [
        [-20, -10, -10, -10, -10, -10, -10, -20],
        [-10, 5, 0, 0, 0, 0, 5, -10],
        [-10, 10, 10, 10, 10, 10, 10, -10],
        [-10, 0, 10, 10, 10, 10, 0, -10],
        [-10, 5, 5, 10, 10, 5, 5, -10],
        [-10, 0, 5, 10, 10, 5, 0, -10],
        [-10, 0, 0, 0, 0, 0, 0, -10],
        [-20, -10, -10, -10, -10, -10, -10, -20]
    ],
        ROOK => [
        [0, 0, 0, 5, 5, 0, 0, 0],
        [-5, 0, 0, 0, 0, 0, 0, -5],
        [-5, 0, 0, 0, 0, 0, 0, -5],
        [-5, 0, 0, 0, 0, 0, 0, -5],
        [-5, 0, 0, 0, 0, 0, 0, -5],
        [-5, 0, 0, 0, 0, 0, 0, -5],
        [5, 10, 10, 10, 10, 10, 10, 5],
        [0, 0, 0, 0, 0, 0, 0, 0]
    ],
        QUEEN => [
        [-20, -10, -10, -5, -5, -10, -10, -20],
        [-10, 0, 5, 0, 0, 0, 0, -10],
        [-10, 5, 5, 5, 5, 5, 0, -10],
        [0, 0, 5, 5, 5, 5, 0, -5],
        [-5, 0, 5, 5, 5, 5, 0, -5],
        [-10, 0, 5, 5, 5, 5, 0, -10],
        [-10, 0, 0, 0, 0, 0, 0, -10],
        [-20, -10, -10, -5, -5, -10, -10, -20]
    ],
        KING => [
        [20, 30, 10, 0, 0, 10, 30, 20],
        [20, 20, 0, 0, 0, 0, 20, 20],
        [-10, -20, -20, -20, -20, -20, -20, -10],
        [-20, -30, -30, -40, -40, -30, -30, -20],
        [-30, -40, -40, -50, -50, -40, -40, -30],
        [-30, -40, -40, -50, -50, -40, -40, -30],
        [-30, -40, -40, -50, -50, -40, -40, -30],
        [-30, -40, -40, -50, -50, -40, -40, -30]
    ]
    )
    const black_king_square_end_game_table::Array{Array{Int8, 1}, 1} = [
        [-50, -30, -30, -30, -30, -30, -30, -50],
        [-30, -30, 0, 0, 0, 0, -30, -30],
        [-30, -10, 20, 30, 30, 20, -10, -30],
        [-30, -10, 30, 40, 40, 30, -10, -30],
        [-30, -10, 30, 40, 40, 30, -10, -30],
        [-30, -10, 20, 30, 30, 20, -10, -30],
        [-30, -20, -10, 0, 0, -10, -20, -30],
        [-50, -40, -30, -20, -20, -30, -40, -50]
    ]
end

In [ ]:
function get_square_value(board::Board, piece::Piece, square::Square, table_opt::String="")::Int
    piece_type::PieceType = ptype(piece)
    if table_opt != "" && is_endgame(board)
        table_opt = "end"
    end

    if Chess.pcolor(piece) == WHITE
        square_table = WHITE_PIECE_SQUARE_TABLES[piece_type]
        if piece_type == KING && table_opt ==  "end"
            square_table = white_king_square_end_game_table
        end
    else
        square_table = BLACK_PIECE_SQUARE_TABLES[piece_type]
        if piece_type == KING && table_opt ==  "end"
            square_table = black_king_square_end_game_table
        end
    end

    squareString::String = tostring(square)
    x::Int8 = Int(squareString[1]) - Int('a')+1
    y::Int8 = parse(Int, squareString[2]) 
    return square_table[9 - y][x]
end


```
function get_square_value(board::Board, piece::Piece, square::Square, table_opt::String="")::Int64
    piece_type::PieceType = ptype(piece)
    if piece_type == PAWN
        square_table = pawn_square_table
    elseif piece_type == KNIGHT
        square_table = knight_square_table
    elseif piece_type == BISHOP
        square_table = bishop_square_table
    elseif piece_type == ROOK
        square_table = rook_square_table
    elseif piece_type == QUEEN
        square_table = queen_square_table
    elseif piece_type == KING
        if table_opt != ""
            if table_opt == "end"
                square_table = king_square_end_game_table
            else
                square_table = king_square_middle_game_table
            end
        elseif is_endgame(board)
            square_table = king_square_end_game_table
        else
            square_table = king_square_middle_game_table
        end
    end
    squareString::String = tostring(square)
    x::Int8 = Int(squareString[1]) - Int('a')+1
    y::Int8 = parse(Int64, squareString[2]) 
    if Chess.pcolor(piece) == BLACK
        square_table = reverse(square_table)
    end
    return square_table[9 - y][x]
end
```

In [ ]:
function calc_score(board::Board, piece::Piece, square::Square)::Int64
    return get_value(piece) + get_square_value(board, piece, square)
end

## Ohne Inkrementelle Evaluation eines Zuges

In [ ]:
function evaluate_position(board::Board)::Int64
    if isterminal(board)
        return terminal_evaluation(board)
    end
    score = 0
    for x in 1:8,y in 1:8
        square = Square(SquareFile(x),SquareRank(y))
        piece = pieceon(board, square)
        if piece != EMPTY
            score += (Chess.pcolor(piece) == WHITE ? 1 : -1) * calc_score(board, piece, square)
        end
    end
    return score 
end

## Inkrementelle Evaluation eines Zuges

### Capture Piece
The function `valueCapturePiece` is used to update the value of a chessboard when a piece is captured by the opponent. As input parameters it takes a chess board (board) and a chess piece that was captured (toMove).

When the function is called, the value of the captured piece is subtracted from the current value of the chess board.

In [ ]:
function valueCapturePiece(board::Board, square::Square)::Int64
    capturePiece = pieceon(board, square)
    return capturePiece == EMPTY ? 0 : calc_score(board, capturePiece, square)
end

In [ ]:
function checkEnPassant(board::Board, toSquare::Square)::Int64
    enpassantSquare = epsquare(board)
    if(enpassantSquare != SQ_NONE && enpassantSquare == toSquare)
        lastToMove = to(lastmove(board))
        return valueCapturePiece(board, lastToMove)
    end 
    return 0
end 

`calc_castle` checks wether the `move` done was a castleing move. 

If no the score will be returned back.

If yes it will adds the value of the move rook. The moved King will be calculated as the regularmove in the `evaluate_move` function. Casteling Kingside does not effect the value of the rook. Casteling Queenside adds +5 to the value of the position.

Check Castle KingSide is ignored, because the score is not changed. The reason is that the rook on square f1/f8 has the value zero. 

In [ ]:
function calc_castle(board::Board, piece::Piece, toSquare::Square)::Int64
    if(ptype(piece) == KING && cancastlequeenside(board, Chess.pcolor(piece)) && file(toSquare) == FILE_C)
        return  5
    end 
    return 0
end

The function evaluate_move(board, move, score) takes in a `board` with it's current static centipawn `score` and a `move` and returns the static centipawn score after doing the move. The function is a incremental implementation meaning it will only consider and calculate the differences between the old and new position.

Following things need to be considered:
- Take away old square position value
- Add new square position value
- Take away score of captured pieces

Special events in chess
- castling
- promotion
- en passant
- entry into endgame

In [ ]:
function evaluate_move(board::Board, move::Move, score::Int64)::Int64
    undoinfo = domove!(board, move)
    if isterminal(board)
        score = terminal_evaluation(board)
        undomove!(board, undoinfo)
        return score
    end
    undomove!(board, undoinfo)
    
    toMove = to(move)
    fromMove = from(move)
    piece = pieceon(board, from(move))
    pieceType = ptype(piece)
    color = Chess.pcolor(piece)
    
    # invert score if piece is black (imitate black is white)
    if color == BLACK
        score = -score
    end
    
    # Sub Points for old Position from Piece
    score -= valueCapturePiece(board, fromMove)
    score += valueCapturePiece(board, toMove)
    # en passant
    if pieceType == PAWN      
        score += checkEnPassant(board, toMove)
    end
    # promotion
    if ispromotion(move)      
        piece = Piece(sidetomove(board), promotion(move))
    end
    
    # castling
    if pieceType == KING     
        score += calc_castle(board, piece, toMove)
    end
    
    score += calc_score(board, piece, toMove)
    
    # invert back
    if color == BLACK
        score = -score
    end
    # entry into endgame
    if !is_endgame(board)
        undoinfo = domove!(board, move)
        if is_endgame(board)
            for kingPos in kings(board)
                k = pieceon(board, kingPos)
                if Chess.pcolor(k) == BLACK
                    score = -score
                end
                score -= get_square_value(board, k, kingPos, "middle")
                score += get_square_value(board, k, kingPos, "end")
                if Chess.pcolor(k) == BLACK
                    score = -score
                end
            end
        end
        undomove!(board, undoinfo)
    end
    
    # entry from endgame to middle
    if is_endgame(board)
        undoinfo = domove!(board, move)
        if !is_endgame(board)
            for kingPos in kings(board)
                k = pieceon(board, kingPos)
                if Chess.pcolor(k) == BLACK
                    score = -score
                end
                score += get_square_value(board, k, kingPos, "middle")
                score -= get_square_value(board, k, kingPos, "end")
                if Chess.pcolor(k) == BLACK
                    score = -score
                end
            end
        end
        undomove!(board, undoinfo)
    end
    return score
end

In [ ]:
function terminal_evaluation(board::Board)::Int64
    if ischeckmate(board)
        color = sidetomove(board)
        if color == WHITE
            return -100000
        else
            return 100000
        end
    elseif isstalemate(board)
        return 0
    elseif ismaterialdraw(board)
        return 0 
    elseif isrule50draw(board) 
        return 0
    end
    return
end